In [0]:
import pyspark
import mlflow
import catboost
import pyspark.sql.functions as F
from sklearn.model_selection import train_test_split

print(mlflow.__version__)

2.22.0


In [0]:
CATALOG = "workspace"
SCHEMA = "safe_driver_prediction"

fs_df = spark.read.table(f"{CATALOG}.{SCHEMA}.feature_store_train")

print(f"training set has {fs_df.count()} rows and {len(fs_df.columns)} columns.")
fs_df.limit(10).display()

training set has 595212 rows and 63 columns.


id,target,ind_01,ind_02_cat,ind_03,ind_04_cat,ind_05_cat,ind_06_bin,ind_07_bin,ind_08_bin,ind_09_bin,ind_10_bin,ind_11_bin,ind_12_bin,ind_13_bin,ind_14,ind_15,ind_16_bin,ind_17_bin,ind_18_bin,reg_01,reg_02,reg_03,car_01_cat,car_02_cat,car_03_cat,car_04_cat,car_05_cat,car_06_cat,car_07_cat,car_08_cat,car_09_cat,car_10_cat,car_11_cat,car_11,car_12,car_13,car_14,car_15,calc_01,calc_02,calc_03,calc_04,calc_05,calc_06,calc_07,calc_08,calc_09,calc_10,calc_11,calc_12,calc_13,calc_14,calc_15_bin,calc_16_bin,calc_17_bin,calc_18_bin,calc_19_bin,calc_20_bin,car_04_cat_simplified,car_11_cat_simplified,ind_02_cat_simplified,ind_05_cat_simplified
1212,0,0,1,7,1,0,1,0,0,0,0,0,0,0,0,12,1,0,0,0.3,0.2,0.8019507466,4,1,0.0,0,1.0,11,1,1,0,1,103,1,0.316227766,0.6671451451,0.3583294573,3.3166247904,0.6,0.3,0.5,1,3,9,3,10,1,4,5,1,6,8,0,1,0,1,0,0,1,0,1,1
1477,0,0,1,7,0,0,1,0,0,0,0,0,0,0,0,3,0,0,1,0.9,0.4,0.7681145748,11,1,0.0,0,0.0,1,1,1,2,1,64,3,0.316227766,0.6603123478,0.3619392214,3.1622776602,0.1,0.2,0.6,3,3,8,3,10,2,6,8,0,4,11,1,1,0,0,0,0,1,0,1,1
4101,0,0,2,0,0,6,1,0,0,0,0,0,0,0,0,0,0,0,1,0.2,0.3,0.8019507466,8,0,0.0,0,1.0,11,1,1,0,1,104,3,0.44721359549999995,0.7809191448,0.4248529157,2.8284271247,0.6,0.7,0.6,3,2,6,3,11,5,9,10,2,2,13,0,1,1,0,0,1,1,1,0,0
7596,0,6,2,9,0,0,0,0,1,0,0,0,0,0,0,7,1,0,0,0.6,1.3,1.0356157589,8,1,1.0,0,1.0,1,1,0,0,1,31,3,0.423792402,1.1132773076,0.4084115571,3.7416573868,0.9,0.8,0.6,2,1,8,4,9,2,3,7,2,2,7,0,1,0,0,1,0,1,0,0,1
9769,0,1,4,7,1,0,1,0,0,0,0,0,0,0,0,5,0,0,0,0.9,0.2,0.8019507466,6,1,0.0,0,0.0,0,1,1,0,1,34,2,0.4,0.7783883541,0.3780211634,3.3166247904,0.5,0.3,0.0,1,4,10,3,11,1,8,8,0,2,11,0,1,0,0,0,0,1,0,0,1
11961,0,2,1,8,0,0,1,0,0,0,0,0,0,0,0,4,0,0,1,0.8,0.9,1.0440306509000001,11,1,1.0,0,1.0,11,1,1,2,1,103,1,0.3160696126,0.5245423034000001,0.2887905816,1.7320508075999999,0.4,0.6,0.8,2,1,7,4,8,2,11,6,1,1,8,0,1,0,0,1,0,1,0,1,1
17080,0,4,1,4,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0.9,0.3,0.7301540933,7,1,0.0,0,0.0,6,0,1,2,1,77,2,0.3741657387,0.7140994444,0.37349698789999997,3.1622776602,0.0,0.5,0.4,0,1,7,1,10,1,5,5,1,2,7,0,0,0,0,0,0,1,0,1,1
20410,0,5,2,6,1,0,0,1,0,0,0,0,0,0,0,7,1,0,0,0.6,0.8,1.6,8,1,1.0,0,1.0,14,1,0,2,1,72,3,0.4,1.1625616485999999,0.35042830939999997,3.6055512755000003,0.2,0.1,0.2,4,3,6,3,11,1,7,5,2,4,2,1,0,0,0,0,0,1,0,0,1
20908,0,0,2,1,1,0,0,1,0,0,0,0,0,0,0,4,1,0,0,0.6,0.5,0.6941721688,7,1,0.0,0,0.0,1,1,0,2,1,80,3,0.316227766,0.8284955997,0.37349698789999997,3.6055512755000003,0.1,0.0,0.8,3,3,8,3,9,0,12,9,0,1,9,0,0,1,0,0,0,1,0,0,1
21142,0,4,1,3,1,0,0,0,1,0,0,0,0,0,0,5,1,0,0,0.4,0.2,0.6982120022,6,1,1.0,0,1.0,14,1,1,0,1,42,2,0.4,0.7369245899,0.3937003937,3.1622776602,0.4,0.8,0.1,4,1,8,3,11,1,9,3,0,4,7,0,0,1,0,1,0,1,0,1,1


In [0]:
X = fs_df.drop("target").toPandas()
y = fs_df.select("target").toPandas()

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=40
)

In [0]:
model = catboost.CatBoostClassifier(
    iterations=1000,
    early_stopping_rounds=100,
    learning_rate=0.05,
    depth=5,
    loss_function="Logloss",
    use_best_model=True,
    random_seed=40
)

In [0]:
model.fit(
    X=X_train, y=y_train, 
    cat_features=None, eval_set=(X_val, y_val), 
    verbose=1
)

0:	learn: 0.6201127	test: 0.6202108	best: 0.6202108 (0)	total: 77.8ms	remaining: 1m 17s
1:	learn: 0.5572407	test: 0.5573323	best: 0.5573323 (1)	total: 153ms	remaining: 1m 16s
2:	learn: 0.5030902	test: 0.5031549	best: 0.5031549 (2)	total: 239ms	remaining: 1m 19s
3:	learn: 0.4565113	test: 0.4565447	best: 0.4565447 (3)	total: 331ms	remaining: 1m 22s
4:	learn: 0.4161545	test: 0.4161762	best: 0.4161762 (4)	total: 398ms	remaining: 1m 19s
5:	learn: 0.3816279	test: 0.3816465	best: 0.3816465 (5)	total: 476ms	remaining: 1m 18s
6:	learn: 0.3517076	test: 0.3517206	best: 0.3517206 (6)	total: 552ms	remaining: 1m 18s
7:	learn: 0.3259450	test: 0.3259733	best: 0.3259733 (7)	total: 623ms	remaining: 1m 17s
8:	learn: 0.3036051	test: 0.3036559	best: 0.3036559 (8)	total: 691ms	remaining: 1m 16s
9:	learn: 0.2846381	test: 0.2846997	best: 0.2846997 (9)	total: 764ms	remaining: 1m 15s
10:	learn: 0.2682200	test: 0.2682989	best: 0.2682989 (10)	total: 833ms	remaining: 1m 14s
11:	learn: 0.2539927	test: 0.2540718	bes

In [0]:
X_test = spark.read.table(f"{CATALOG}.{SCHEMA}.feature_store_test")
X_test = X_test.toPandas()

In [0]:
preds = model.predict_proba(X_test)[:, 1]
preds_df = pd.DataFrame(
    {"id": X_test["id"], "target": preds}
).sort_values("id")

preds_df.to_csv(
    f"/Volumes/{CATALOG}/{SCHEMA}/kaggle_competition_data/submission_v0.2.csv",
    index=False
)